###### Import of the Mexican YouTube video csv file and category json files.
* For the csv file a ISO-8859-1 codec was needed as the more universal UFT-8 codec resulted in an unexpected end of data error - see below for more details.
* A regular store of the JSON data into a DataFrame resulted in "missing" data as the JSON was not fully traversed. As a result an alternate approach was used to parse out those non-traversed data elements into list and applied to a dataframe.
* In order to merge the csv and JSON dataframes on category id an integer conversion was made on the id_list in the JSON dataframe. This prevented issues/errors with nt64 and object columns when attempting the merge. The dataframes were "left" joined on the csv dataframe.
* Added in a country code of 'MX' to the combined dataframe to distinguish a country of origin.

In [4]:
import pandas as pd
from sqlalchemy import create_engine
import json
import os

In [5]:
# Could not use 'utf-8' codec otherwise error "can't decode byte 0xc3 in position 254: unexpected end of data"
csv_file = "./Resources/MXvideos.csv"
MXvideos_df = pd.read_csv(csv_file, encoding='ISO-8859-1' )
MXvideos_df.head()

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
0,SbOwzAl9ZfQ,17.14.11,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,24,2017-11-13T06:06:22.000Z,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,https://i.ytimg.com/vi/SbOwzAl9ZfQ/default.jpg,False,False,False,Disfruta la presencia del Chef Torreblanca en ...
1,klOV6Xh-DnI,17.14.11,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,2017-11-13T05:11:58.000Z,La Voz Mexico 7,104972,271,174,369,https://i.ytimg.com/vi/klOV6Xh-DnI/default.jpg,False,False,False,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...
2,6L2ZF7Qzsbk,17.14.11,LOUIS CKAGÃ - EL PULSO DE LA REPÃBLICA,El Pulso De La RepÃºblica,25,2017-11-13T17:00:02.000Z,"Chumel Torres|""El Pulso de la Republica""|""noti...",136064,10105,266,607,https://i.ytimg.com/vi/6L2ZF7Qzsbk/default.jpg,False,False,False,La canciÃ³n del principio se llama âEste esp...
3,hcY52MFWMDM,17.14.11,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,25,2017-11-13T03:47:10.000Z,"temblor|""costa rica""|""sismo en costa rica""",96153,378,171,208,https://i.ytimg.com/vi/hcY52MFWMDM/default.jpg,False,False,False,El video es de un Walmart en el pais centroame...
4,_OXDcGPVAa4,17.14.11,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,26,2017-11-13T19:17:48.000Z,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs...",499965,57781,681,7428,https://i.ytimg.com/vi/_OXDcGPVAa4/default.jpg,False,False,False,MI HERMANO NARRA MI RUTINA DE MAQUILLAJE\nhttp...


In [6]:
#removed three columns as a part of the clean up activities (publish_time,thumbnail_link,description)
new_MXvideos_df= MXvideos_df[["video_id","trending_date","title","channel_title","category_id","tags","views","likes","dislikes","comment_count","comments_disabled","ratings_disabled","video_error_or_removed"]].copy()
new_MXvideos_df

,video_id,trending_date,title,channel_title,category_id,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed
0,SbOwzAl9ZfQ,17.14.11,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,24,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,False,False,False
1,klOV6Xh-DnI,17.14.11,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,La Voz Mexico 7,104972,271,174,369,False,False,False
2,6L2ZF7Qzsbk,17.14.11,LOUIS CKAGÃ - EL PULSO DE LA REPÃBLICA,El Pulso De La RepÃºblica,25,"Chumel Torres|""El Pulso de la Republica""|""noti...",136064,10105,266,607,False,False,False
3,hcY52MFWMDM,17.14.11,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,25,"temblor|""costa rica""|""sismo en costa rica""",96153,378,171,208,False,False,False
4,_OXDcGPVAa4,17.14.11,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,26,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs...",499965,57781,681,7428,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40446,r63VBOagGAo,18.14.06,Shawn Mendes x Portugal (FPF Official World Cu...,FPFutebolOficial,17,"Canal|""Oficial""|""da""|""FederaÃ§Ã£o""|""Portuguesa...",653114,65639,547,3826,False,False,False
40447,i7r_kMbyngk,18.14.06,AMLO llegÃ³ con su esposa al Tercer Debate en ...,AMX,25,"AMLO|""morena""|""polÃ­tica""|""PRI""|""PAN""|""PRD""|""L...",121674,3558,58,491,False,False,False
40448,_jnwjdMe3Zo,18.14.06,Maire usa una blusa kawaiii Â¿adorable o ridÃ­...,Historias Wink,24,"maire|""maire vs el internet""|""maire vs interne...",14067,538,6,90,False,False,False
40449,pAH9omNAWA4,18.14.06,La Jefa del CampeÃ³n - CapÃ­tulo 2 Parte 3/4,Tele Time,22,[none],22898,94,11,3,False,False,False


In [7]:
#Store JSON data into a DataFrame
json_file = "./Resources/MX_category_id.json"
MX_category_df=pd.read_json(json_file)
MX_category_df.head()

,kind,etag,items
0,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""XI..."
1,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""XI..."
2,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""XI..."
3,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""XI..."
4,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/1v2mrzYSYG6onNLt2...","{'kind': 'youtube#videoCategory', 'etag': '""XI..."


In [8]:
with open("./Resources/MX_category_id.json") as data_file:
    data = json.load(data_file) 

#    kind_list = []
    etag_list = []
    id_list = []
    title_list = []
#    data_list = []

# You will need to change the range size depending on your file

    for i in range(31):
    # data_list.append(data["kind"])  If you want the header before the list of dictionaries
    # data_list.append(data["etag"])  If you want the header before the list of dictionaries
        try:
#            kind_list.append(data['items'][i]['kind'])
            etag_list.append(data['items'][i]['etag'])
            id_list.append(data['items'][i]['id'])
            title_list.append(data['items'][i]['snippet']['title'])
        except KeyError:
#            kind_list.append("")
            etag_list.append("")
            id_list.append("")
            title_list.append("")
            
#print("------------ kind list ------------")    
#print(kind_list) 
print("------------ etag list -------------")
print(etag_list) 
print("------------ id list ---------------")
print(id_list) 
print("------------ title list ------------")
print(title_list) 
print("------------ finished --------------")

        

------------ etag list -------------
['"XI7nbFXulYBIpL0ayR_gDh3eu1k/Xy1mB4_yLrHy_BmKmPBggty2mZQ"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/UZ1oLIIz2dxIhO45ZTFR3a3NyTA"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/nqRIq97-xe5XRZTxbknKFVe5Lmg"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/HwXKamM1Q20q9BN-oBJavSGkfDI"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/9GQMSRjrZdHeb1OEM1XVQ9zbGec"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/FJwVpGCVZ1yiJrqZbpqe68Sy_OE"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/M-3iD9dwK7YJCafRf_DkLN8CouA"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/WmA0qYEfjWsAoyJFSw2zinhn2wM"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/EapFaGYG7K0StIXVf8aba249tdM"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/xId8RX7vRN8rqkbYZbNIytUQDRo"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/G9LHzQmx44rX2S5yaga_Aqtwz8M"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/UVB9oxX2Bvqa_w_y3vXSLVK5E_s"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/QiLK0ZIrFoORdk_g2l_XR_ECjDc"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/r6Ck6Z0_L0rG37VJQR200SGNA_w"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/EoYkczo9I3RCf96RveKTOgOPkUM"', '"XI7nbFXulYBIpL0ayR_gDh3eu1k/w5HjcTD82G_XA3xBc

In [9]:
# Please note that I did not change the id in JSON file until now when added to the existing dataframe
#MX_category_df["kind"] = kind_list
MX_category_df["etag"] = etag_list
MX_category_df["category_id"] = id_list
MX_category_df["title"] = title_list

print(MX_category_df)

                                 kind  \
0   youtube#videoCategoryListResponse   
1   youtube#videoCategoryListResponse   
2   youtube#videoCategoryListResponse   
3   youtube#videoCategoryListResponse   
4   youtube#videoCategoryListResponse   
5   youtube#videoCategoryListResponse   
6   youtube#videoCategoryListResponse   
7   youtube#videoCategoryListResponse   
8   youtube#videoCategoryListResponse   
9   youtube#videoCategoryListResponse   
10  youtube#videoCategoryListResponse   
11  youtube#videoCategoryListResponse   
12  youtube#videoCategoryListResponse   
13  youtube#videoCategoryListResponse   
14  youtube#videoCategoryListResponse   
15  youtube#videoCategoryListResponse   
16  youtube#videoCategoryListResponse   
17  youtube#videoCategoryListResponse   
18  youtube#videoCategoryListResponse   
19  youtube#videoCategoryListResponse   
20  youtube#videoCategoryListResponse   
21  youtube#videoCategoryListResponse   
22  youtube#videoCategoryListResponse   
23  youtube#vide

In [10]:
# Make the id_list an integer to avoid issues with nt64 and object columns when attempting merge.
MX_category_df["category_id"] = MX_category_df["category_id"].astype(int)

In [11]:
MX_category_df

,kind,etag,items,category_id,title
0,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/Xy1mB4_yLrHy_BmKm...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",1,Film & Animation
1,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/UZ1oLIIz2dxIhO45Z...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",2,Autos & Vehicles
2,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/nqRIq97-xe5XRZTxb...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",10,Music
3,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/HwXKamM1Q20q9BN-o...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",15,Pets & Animals
4,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/9GQMSRjrZdHeb1OEM...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",17,Sports
5,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/FJwVpGCVZ1yiJrqZb...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",18,Short Movies
6,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/M-3iD9dwK7YJCafRf...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",19,Travel & Events
7,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/WmA0qYEfjWsAoyJFS...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",20,Gaming
8,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/EapFaGYG7K0StIXVf...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",21,Videoblogging
9,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",22,People & Blogs


In [12]:
combined_MXdata=pd.merge(new_MXvideos_df,MX_category_df, on= "category_id", how = "left")
combined_MXdata

,video_id,trending_date,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,kind,etag,items,title_y
0,SbOwzAl9ZfQ,17.14.11,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,24,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Entertainment
1,klOV6Xh-DnI,17.14.11,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,La Voz Mexico 7,104972,271,174,369,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",People & Blogs
2,6L2ZF7Qzsbk,17.14.11,LOUIS CKAGÃ - EL PULSO DE LA REPÃBLICA,El Pulso De La RepÃºblica,25,"Chumel Torres|""El Pulso de la Republica""|""noti...",136064,10105,266,607,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/QiLK0ZIrFoORdk_g2...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",News & Politics
3,hcY52MFWMDM,17.14.11,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,25,"temblor|""costa rica""|""sismo en costa rica""",96153,378,171,208,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/QiLK0ZIrFoORdk_g2...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",News & Politics
4,_OXDcGPVAa4,17.14.11,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,26,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs...",499965,57781,681,7428,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/r6Ck6Z0_L0rG37VJQ...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Howto & Style
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40446,r63VBOagGAo,18.14.06,Shawn Mendes x Portugal (FPF Official World Cu...,FPFutebolOficial,17,"Canal|""Oficial""|""da""|""FederaÃ§Ã£o""|""Portuguesa...",653114,65639,547,3826,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/9GQMSRjrZdHeb1OEM...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Sports
40447,i7r_kMbyngk,18.14.06,AMLO llegÃ³ con su esposa al Tercer Debate en ...,AMX,25,"AMLO|""morena""|""polÃ­tica""|""PRI""|""PAN""|""PRD""|""L...",121674,3558,58,491,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/QiLK0ZIrFoORdk_g2...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",News & Politics
40448,_jnwjdMe3Zo,18.14.06,Maire usa una blusa kawaiii Â¿adorable o ridÃ­...,Historias Wink,24,"maire|""maire vs el internet""|""maire vs interne...",14067,538,6,90,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Entertainment
40449,pAH9omNAWA4,18.14.06,La Jefa del CampeÃ³n - CapÃ­tulo 2 Parte 3/4,Tele Time,22,[none],22898,94,11,3,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",People & Blogs


In [13]:
print(combined_MXdata)

          video_id trending_date  \
0      SbOwzAl9ZfQ      17.14.11   
1      klOV6Xh-DnI      17.14.11   
2      6L2ZF7Qzsbk      17.14.11   
3      hcY52MFWMDM      17.14.11   
4      _OXDcGPVAa4      17.14.11   
...            ...           ...   
40446  r63VBOagGAo      18.14.06   
40447  i7r_kMbyngk      18.14.06   
40448  _jnwjdMe3Zo      18.14.06   
40449  pAH9omNAWA4      18.14.06   
40450  dj5Z4jTE3-c      18.14.06   

                                                 title_x  \
0                         CapÃ­tulo 12 | MasterChef 2017   
1      ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...   
2               LOUIS CKAGÃ - EL PULSO DE LA REPÃBLICA   
3       Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017   
4               DOG HACKS | MUSAS LESSLIE LOS POLINESIOS   
...                                                  ...   
40446  Shawn Mendes x Portugal (FPF Official World Cu...   
40447  AMLO llegÃ³ con su esposa al Tercer Debate en ...   
40448  Maire usa una blusa 

In [14]:
# Add in the country code to the combined  dataframe
combined_MXdata['country']='MX'

In [15]:
display(combined_MXdata)

,video_id,trending_date,title_x,channel_title,category_id,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,kind,etag,items,title_y,country
0,SbOwzAl9ZfQ,17.14.11,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,24,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Entertainment,MX
1,klOV6Xh-DnI,17.14.11,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,La Voz Mexico 7,104972,271,174,369,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",People & Blogs,MX
2,6L2ZF7Qzsbk,17.14.11,LOUIS CKAGÃ - EL PULSO DE LA REPÃBLICA,El Pulso De La RepÃºblica,25,"Chumel Torres|""El Pulso de la Republica""|""noti...",136064,10105,266,607,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/QiLK0ZIrFoORdk_g2...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",News & Politics,MX
3,hcY52MFWMDM,17.14.11,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,25,"temblor|""costa rica""|""sismo en costa rica""",96153,378,171,208,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/QiLK0ZIrFoORdk_g2...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",News & Politics,MX
4,_OXDcGPVAa4,17.14.11,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,26,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs...",499965,57781,681,7428,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/r6Ck6Z0_L0rG37VJQ...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Howto & Style,MX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40446,r63VBOagGAo,18.14.06,Shawn Mendes x Portugal (FPF Official World Cu...,FPFutebolOficial,17,"Canal|""Oficial""|""da""|""FederaÃ§Ã£o""|""Portuguesa...",653114,65639,547,3826,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/9GQMSRjrZdHeb1OEM...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Sports,MX
40447,i7r_kMbyngk,18.14.06,AMLO llegÃ³ con su esposa al Tercer Debate en ...,AMX,25,"AMLO|""morena""|""polÃ­tica""|""PRI""|""PAN""|""PRD""|""L...",121674,3558,58,491,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/QiLK0ZIrFoORdk_g2...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",News & Politics,MX
40448,_jnwjdMe3Zo,18.14.06,Maire usa una blusa kawaiii Â¿adorable o ridÃ­...,Historias Wink,24,"maire|""maire vs el internet""|""maire vs interne...",14067,538,6,90,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Entertainment,MX
40449,pAH9omNAWA4,18.14.06,La Jefa del CampeÃ³n - CapÃ­tulo 2 Parte 3/4,Tele Time,22,[none],22898,94,11,3,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",People & Blogs,MX


In [16]:
renamed_combined_MXdata = combined_MXdata.rename(columns={"title_x": "video_title",
                                                          "title_y": "category_title"})
renamed_combined_MXdata

,video_id,trending_date,video_title,channel_title,category_id,tags,views,likes,dislikes,comment_count,comments_disabled,ratings_disabled,video_error_or_removed,kind,etag,items,category_title,country
0,SbOwzAl9ZfQ,17.14.11,CapÃ­tulo 12 | MasterChef 2017,MasterChef 2017,24,"MasterChef Junior 2017|""TV Azteca""|""recetas""|""...",310130,4182,361,1836,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Entertainment,MX
1,klOV6Xh-DnI,17.14.11,ALEXA EX-INTEGRANTE DEL GRUPO TIMBIRICHE RENUN...,Micky Contreras Martinez,22,La Voz Mexico 7,104972,271,174,369,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",People & Blogs,MX
2,6L2ZF7Qzsbk,17.14.11,LOUIS CKAGÃ - EL PULSO DE LA REPÃBLICA,El Pulso De La RepÃºblica,25,"Chumel Torres|""El Pulso de la Republica""|""noti...",136064,10105,266,607,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/QiLK0ZIrFoORdk_g2...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",News & Politics,MX
3,hcY52MFWMDM,17.14.11,Sismo de 6.7 sacude Costa Rica 12 Noviembre 2017,Casanare,25,"temblor|""costa rica""|""sismo en costa rica""",96153,378,171,208,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/QiLK0ZIrFoORdk_g2...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",News & Politics,MX
4,_OXDcGPVAa4,17.14.11,DOG HACKS | MUSAS LESSLIE LOS POLINESIOS,Musas,26,"MUSAS|""lesslie""|""karen""|""hacks""|""perros""|""dogs...",499965,57781,681,7428,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/r6Ck6Z0_L0rG37VJQ...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Howto & Style,MX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40446,r63VBOagGAo,18.14.06,Shawn Mendes x Portugal (FPF Official World Cu...,FPFutebolOficial,17,"Canal|""Oficial""|""da""|""FederaÃ§Ã£o""|""Portuguesa...",653114,65639,547,3826,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/9GQMSRjrZdHeb1OEM...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Sports,MX
40447,i7r_kMbyngk,18.14.06,AMLO llegÃ³ con su esposa al Tercer Debate en ...,AMX,25,"AMLO|""morena""|""polÃ­tica""|""PRI""|""PAN""|""PRD""|""L...",121674,3558,58,491,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/QiLK0ZIrFoORdk_g2...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",News & Politics,MX
40448,_jnwjdMe3Zo,18.14.06,Maire usa una blusa kawaiii Â¿adorable o ridÃ­...,Historias Wink,24,"maire|""maire vs el internet""|""maire vs interne...",14067,538,6,90,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/UVB9oxX2Bvqa_w_y3...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",Entertainment,MX
40449,pAH9omNAWA4,18.14.06,La Jefa del CampeÃ³n - CapÃ­tulo 2 Parte 3/4,Tele Time,22,[none],22898,94,11,3,False,False,False,youtube#videoCategoryListResponse,"""XI7nbFXulYBIpL0ayR_gDh3eu1k/xId8RX7vRN8rqkbYZ...","{'kind': 'youtube#videoCategory', 'etag': '""XI...",People & Blogs,MX


In [21]:
renamed_combined_MXdata.to_csv("Resources/cleanedMXdata.csv", index=False)